In [ ]:
import os
import json
from tapeagents.llms import OpenrouterLLM
from dotenv import load_dotenv
from func_util import income_eligible
from func_util import ai_employment_eligible
from func_util import get_subsidy_coverage
from func_util import run_poverty_agent
from func_util import run_decision_agent
from tapeagents.environment import ToolCollectionEnvironment
from tapeagents.tools.web_search import WebSearch
from tapeagents.tools.calculator import Calculator
from tapeagents.tools.simple_browser import SimpleBrowser
from tapeagents.nodes import StandardNode, Stop
from tapeagents.agent import Agent
from tapeagents.dialog_tape import DialogTape, UserStep
from tapeagents.orchestrator import main_loop
from IPython.display import display, HTML, Markdown
load_dotenv()

#LLM
open_router_api_key = os.environ["OPEN_ROUTER_API_KEY"]
llm = OpenrouterLLM(
    model_name="meta-llama/llama-3.3-70b-instruct:free",  # https://openrouter.ai/meta-llama/llama-3.3-70b-instruct:free
    api_token=open_router_api_key,
    parameters={"temperature": 0.1},
)

#Environment setting


environment = ToolCollectionEnvironment(tools=[WebSearch(), SimpleBrowser(), Calculator()])
environment.initialize()

# ---Incoming research agent---


system_prompt = "You are an assistant that can use web search to answer questions."

search_agent = Agent.create(
    llms=llm,
    nodes=[
        StandardNode(
            name="think",
            system_prompt=system_prompt,
            guidance="Let's think about how to use available tools to answer the user's question. Do not answer the question yet.",
        ),
        StandardNode(
            name="act",
            system_prompt=system_prompt,
            guidance="Call the proposed tool.",
            use_known_actions=True,
            use_function_calls=True,
        ),
        StandardNode(
            name="answer",
            system_prompt=system_prompt,
            guidance="Give a final answer to the user's question based on the tool result.",
        ),
        Stop(),
    ],
    known_actions=environment.actions(),
    tools_description=environment.tools_description()
)

# --- Poverty Level Agent ---

poverty_system_prompt = (
    "You are an assistant that assesses the poverty level of a household for subsidy eligibility. "
    "Consider the following fields: household_income, housing_situation, partner_employed, num_children. "
    "Output only one word: low, medium, or high."
)

poverty_agent = Agent.create(
    llms=llm,
    nodes=[
        StandardNode(
            name="think",
            system_prompt=poverty_system_prompt,
            guidance="Think about how to assess the poverty level using the provided data. Do not answer yet.",
        ),
        StandardNode(
            name="answer",
            system_prompt=poverty_system_prompt,
            guidance="Based on the data, output only the poverty level: low, medium, or high.",
        ),
        Stop(),
    ],
    known_actions=environment.actions(),
    tools_description=environment.tools_description()
)

#----DECISION AGENT----
decision_system_prompt = (
    "You are an assistant that makes final decisions on subsidy applications. "
    "Given the poverty level, child ages, partner employment, and application flags, "
    "If sensitive flag exists 'Specific human view: yes', the percentage of Based on the above, find the rate this application result in form of 'Accepted rate: x% \n Decline rate: y% \n Specific human view: yes/no' each stats in new line at and show firstly.Then, on new next line, explain decision shortly in 3-5 sentence"
)

decision_agent = Agent.create(
    llms=llm,
    nodes=[
        StandardNode(
            name="think",
            system_prompt=decision_system_prompt,
            guidance="Think about the decision based on the provided data. Do not answer yet.",
        ),
        StandardNode(
            name="answer",
            system_prompt=decision_system_prompt,
            guidance="If sensitive flag exists 'Specific human view: yes', the percentage of Based on the above, find the rate this application result in form of 'Accepted rate: x% \n Decline rate: y% \n Specific human view: yes/no' each stats in new line at and show firstly.Then on new next line, explain decision shortly in 3-5 sentence",
        ),
        Stop(),
    ],
    known_actions=environment.actions(),
    tools_description=environment.tools_description()
)

#USE AGENT


user_question = "Using the most recent CBS data for Dutch municipalities (2023), estimate the 75th percentile (upper quartile) of standardized disposable household income in Amsterdam, given that the median income is approximately €30,100 and national income distribution suggests the 75th percentile is about 45% higher than the median. Give me only nothing other than number of result in euros"
tape = DialogTape(steps=[UserStep(content=user_question)])
final_tape1 = None

for event in main_loop(search_agent, tape, environment):
    if event.agent_event and event.agent_event.step:
        step = event.agent_event.step
        print(f"Agent step ({step.metadata.node}):\n{step.llm_view()}\n---")
    elif event.agent_tape:
        tape = event.agent_tape

agent_answer = tape[-2].reasoning 
upper_quartile = int(agent_answer.replace(",", ""))

styled_html = f"""
<div style="border:2px solid #4CAF50; border-radius:8px; padding:16px; background:#f9fff9; font-family:sans-serif; width:fit-content;">
    <h2 style="color:#4CAF50;">Agent1 Earnings Threshold Report</h2>
    <p style="font-size:1.2em;"><b>Upper Quartile:</b> <span style="color:#333;">€{upper_quartile:,}</span></p>
    <p style="font-size:1em; color:#555;">Raw agent answer: <code>{agent_answer}</code></p>
</div>
"""
display(HTML(styled_html))


#application input
with open("../data/application.json", "r") as f:
    data = json.load(f)
app_id = input("Enter the application ID to check (e.g., A001): ").strip()
application = next((app for app in data if app.get("application_id") == app_id), None)

# CHECK ELIGIBLE CONDITION
if application:
    if income_eligible(agent_answer, application):
        display(Markdown("**✅ Eligible:** Household income is below the upper quartile."))

        result = ai_employment_eligible(application)
        display(Markdown(f"**🤖 AI_check_employment_eligible:** `{result}`"))

        display(Markdown(
            f"**📋 Please recheck:**  \n"
            f"• Applicant employment status: `{application.get('employment_status', '')}`  \n"
            f"• Partner employed: `{application.get('partner_employed', False)}`"
        ))

        coverage = get_subsidy_coverage(application, upper_quartile)
        display(Markdown(f"**💰 Max subsidy coverage:** `{coverage}%`"))



        #POVERTY AGENT
        agent_answer2 = run_poverty_agent(poverty_agent, application, environment, verbose=True)

        #DECISION AGENT
        agent_answer3 = run_decision_agent(decision_agent, application, agent_answer2, environment, verbose=True)

        styled_html = f"""
        <div style="border:2px solid #2196F3; border-radius:8px; padding:16px; background:#f0f8ff; font-family:sans-serif; width:fit-content; margin-bottom: 16px;">
            <h2 style="color:#2196F3;">Agent2 Poverty Report</h2>
            <p style="font-size:1.2em;"><b>Poverty Level:</b> <span style="color:#333;">{agent_answer2}</span></p>
        </div>
        <div style="border:2px solid #FF9800; border-radius:8px; padding:16px; background:#fff8e1; font-family:sans-serif; width:fit-content;">
            <h2 style="color:#FF9800;">Agent3 Decision Report</h2>
            <p style="font-size:1.2em;"><b>Decision:</b> <span style="color:#333;">{agent_answer3}</span></p>
        </div>
        """

        display(HTML(styled_html))

    else:
        print("Not eligible: Household income exceeds the upper quartile.")
else:
    print("Application ID not found.")

/home/dieppham/personal/hack4her/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Agent step (think):
To answer the question, we can follow these steps:

1. **Understand the concept of the 75th percentile**: The 75th percentile, also known as the upper quartile (Q3), is the value below which 75% of the data points fall. In the context of income, it means that 75% of the households have an income below this value, and 25% have an income above it.

2. **Use the given median income**: We are given that the median income in Amsterdam is approximately €30,100. The median is the middle value in a list of numbers, which is also the 50th percentile.

3. **Apply the national income distribution guideline**: We're told that nationally, the 75th percentile is about 45% higher than the median. Assuming this guideline applies to Amsterdam as well (since specific data for Amsterdam is not provided), we can calculate the 75th percentile of Amsterdam's income by increasing the median by 45%.

4. **Calculate the 75th percentile**: To find the 75th percentile, we calculate 45% of the

Failed to parse LLM output dict: 43645.0

Error: 'float' object is not iterable
Traceback (most recent call last):
  File "/home/dieppham/personal/hack4her/tapeagents/nodes.py", line 409, in parse_completion
    steps = [TypeAdapter(self._steps_type).validate_python(step_dict) for step_dict in step_dicts]
TypeError: 'float' object is not iterable


Agent step (answer):
{
  "kind": "llm_output_parsing_failure_action",
  "error": "Failed to parse LLM output dict: 'float' object is not iterable",
  "llm_output": "43645.0"
}
---
Agent step (answer):
{
  "kind": "set_next_node",
  "next_node": "answer"
}
---
Agent step (answer):
43,645
---
Agent step (Stop):
{
  "kind": "final_step",
  "reason": ""
}
---


KeyboardInterrupt: Interrupted by user